## Initializing the Local Model

In [1]:
from langchain_community.llms import Ollama 
llm = Ollama(model="mistral", temperature = 0)

In [2]:
llm.invoke("What is LangGraph?")

" LangGraph is a large-scale graph database specifically designed for natural language processing (NLP) and information retrieval tasks. It was developed by Microsoft Research and is part of the Microsoft Open Data Initiative.\n\nLangGraph aims to provide a more efficient way to store, query, and process large amounts of text data compared to traditional relational databases or file-based storage systems. By representing text data as graphs, LangGraph enables advanced NLP tasks such as entity recognition, relation extraction, and semantic similarity analysis to be performed directly on the graph, rather than requiring separate preprocessing steps and external tools.\n\nLangGraph's key features include:\n\n1. Scalability: It can handle large volumes of data and complex queries with high performance.\n2. Flexibility: It supports various data models, including property graphs and RDF triples, allowing for diverse use cases.\n3. Integration: It integrates well with other Microsoft Azure se

## Indexing: Load

In [3]:
from langchain_community.document_loaders import WebBaseLoader 

loader = WebBaseLoader(
    web_path=("https://blog.langchain.dev/langgraph/"),
)
docs= loader.load()

In [4]:
docs[0].page_content

'\n\n\nLangGraph\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nSkip to content\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nBy LangChain\n\n\n\n\nRelease Notes\n\n\n\n\nCase Studies\n\n\n\n\nLangChain\n\n\n\n\nGitHub\n\n\n\n\nDocs\n\n\n\n\n\nSign in\nSubscribe\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\n\nLangGraph\n\nBy LangChain\n7 min read\nJan 17, 2024\n\n\n\n\n\nTL;DR: LangGraph is module built on top of LangChain to better enable creation of cyclical graphs, often needed for agent runtimes.Python RepoPython YouTube PlaylistJS RepoIntroductionOne of the things we highlighted in our LangChain v0.1 announcement was the introduction of a new library: LangGraph. LangGraph is built on top of LangChain and completely interoperable with the LangChain ecosystem. It adds new value primarily through the introduction of an easy way to create cyclical graphs. This is often useful when creating agent runtimes.In this blog post, we will first walk through t

In [5]:
len(docs[0].page_content)

12341

## Indexing: Split

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter= RecursiveCharacterTextSplitter(
    chunk_size = 1000,
    chunk_overlap = 200,
    add_start_index = True,
)

In [7]:
all_splits = text_splitter.split_documents(docs)

In [8]:
len(all_splits)

21

In [9]:
len(all_splits[1].page_content)

997

## Indexing: Store

In [10]:
from langchain_community.embeddings import OllamaEmbeddings

embedding = OllamaEmbeddings(
    model="nomic-embed-text",
)

In [11]:
from langchain_community.vectorstores import Chroma

vectorstore = Chroma.from_documents(
    documents = all_splits,
    embedding = embedding,
)

In [12]:
retriever = vectorstore.as_retriever(
    search_type = "similarity",
    search_kwargs = {"k":6}
)

In [13]:
retriever.get_relevant_documents(
    "what is LangGraph?"
)

[Document(page_content='"model",\n    should_continue,\n    {\n        "end": END,\n        "continue": "tools"\n    }\n)CompileAfter we define our graph, we can compile it into a runnable! This simply takes the graph definition we\'ve created so far an returns a runnable. This runnable exposes all the same method as LangChain runnables (.invoke, .stream, .astream_log, etc) allowing it to be called in the same manner as a chain.app = graph.compile()Agent ExecutorWe\'ve recreated the canonical LangChain AgentExecutor with LangGraph. This will allow you to use existing LangChain agents, but allow you to more easily modify the internals of the AgentExecutor. The state of this graph by default contains concepts that should be familiar to you if you\'ve used LangChain agents: input, chat_history, intermediate_steps (and agent_outcome to represent the most recent agent outcome)from typing import TypedDict, Annotated, List, Union\nfrom langchain_core.agents import AgentAction, AgentFinish', m

## Retrieval and Generation: Generate

In [14]:
from langchain import hub 

prompt= hub.pull("rlm/rag-prompt")

In [15]:
def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

In [16]:
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt 
    | llm 
    | StrOutputParser()
)

In [17]:
rag_chain.invoke("What is LangGraph?")

' LangGraph is a Python module built on top of LangChain, providing easier creation of cyclical graphs for agent runtimes. It enables more control over agent flow and the calling of tools.'

In [18]:
rag_chain.invoke("What is LangGraph used for?")

' LangGraph is a module developed on top of LangChain, enhancing its capabilities for creating cyclical graphs, particularly useful for agent runtimes. It allows the creation of nodes and edges with different types such as starting edges, normal edges, and conditional edges. The motivation behind LangGraph is to provide an easier way to build complex agent runtime graphs.'

Thanks for reading 😀

Let's connect [YouTube](http://youtube.com/tirendazacademy) | [Medium](http://tirendazacademy.medium.com) | [X](http://x.com/tirendazacademy) | [Linkedin](https://www.linkedin.com/in/tirendaz-academy) 😎